In [1]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions
from keras.preprocessing import image
from keras.optimizers import SGD
from keras.layers import Dense, GlobalAveragePooling2D, merge
from keras import backend as K
import numpy as np
from keras.callbacks import Callback
from sklearn import preprocessing
from keras.utils.np_utils import to_categorical

import preprocess as ppx

Using TensorFlow backend.


In [2]:
# a képek betöltése és előfeldolgozása
image_file_names, data = ppx.csv_load()
image_name_dict, images = ppx.load_images(image_file_names)
train_images, train_data = ppx.data_preprocess(images, data, image_name_dict)

In [5]:
train_images.shape

(14, 256, 256, 3)

In [4]:
# adatok megfelelő formátumra hozása a keras számára
labels = np.array([ls[1] for ls in train_data])

encoder = preprocessing.LabelEncoder()
encoder.fit(labels)

encoded_l = encoder.transform(labels)
print(encoded_l)

labels_onehot = to_categorical(encoded_l)
print(labels_onehot)

[13  9  0  5  6  7  8 10  1  4 11 12  2  3]
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [5]:
class TrainingHistory(Callback):
    # Tanulási folyamat elején létrehozunk egy-egy üres listát a kinyerni kívánt metrikák tárolása céljából.
    def on_train_begin(self, logs={}):
        # Hiba mértéke a tanító adatokon.
        self.losses = []
        # Hiba mértéke a validációs adatokon.
        self.valid_losses = []
        # A modell jóságát, pontosságát mérő mutatószám a tanító adatokon. 
        self.accs = []
        # A modell jóságát, pontosságát mérő mutatószám a validációs adatokon. 
        self.valid_accs = []
        # A tanítási fázisok sorszámozása.
        self.epoch = 0
    
    # Minden egyes tanítási fázis végén mentsük el, hogy hogyan teljesít aktuálisan a háló. 
    def on_epoch_end(self, epoch, logs={}):
        if epoch % 1 == 0:
            self.losses.append(logs.get('loss'))
            self.valid_losses.append(logs.get('val_loss'))
            self.accs.append(logs.get('acc'))
            self.valid_accs.append(logs.get('val_acc'))
            self.epoch += 1
            
history = TrainingHistory()

In [6]:
##########################################################
# error esetére, elvileg nem okoz gondot 'jó' esetben sem
import tensorflow as tf
tf.python.control_flow_ops = tf
##########################################################

# előtanított modell betöltése, a fully-connected rétegek nélkül
base_model = InceptionV3(weights='imagenet', include_top=False)
# az utolsó konvolúciós réteg utána egy global average pooling réteget teszünk, ez rögtön "lapítja" (flatten) a 2D konvolúciót


In [15]:
# kinyerjük a stílusjegyeket a cnn köztes rétegegeiből (és max pool cnn kimeneti rétegére)
style1 = base_model.layers[54 ].output
style1 = GlobalAveragePooling2D()(style1)
style1 = Dense(96, activation='relu')(style1)
style2 = base_model.layers[117].output
style2 = GlobalAveragePooling2D()(style2)
style2 = Dense(160, activation='relu')(style2)
style3 = base_model.layers[184].output
style3 = GlobalAveragePooling2D()(style3)
style3 = Dense(320, activation='relu')(style3)

style_final = base_model.output
style_final = GlobalAveragePooling2D()(style_final)

# egymás mellé tesszük a különböző szintű feature-öket
ff = merge([style1, style2, style3, style_final], mode='concat')

# ezután hozzáadunk két előrecsatolt réteget ReLU aktivációs függvénnyel
ff = Dense(1024, activation='relu')(ff)
ff = Dense(1024, activation='relu')(ff)

# és végül egy kimenete lesz a hálónak - a "binary_crossentropy" költségfüggvénynek erre van szüksége
predictions = Dense(labels_onehot.shape[1], activation='softmax')(ff)
# a model létrehozása
model = Model(input=base_model.input, output=predictions)

tf
tf
tf
tf


In [16]:
# két lépésben fogjuk tanítani a hálót
# az első lépésben csak az előrecsatolt rétegeket tanítjuk, a konvolúciós rétegeket befagyasztjuk
for layer in base_model.layers:
    layer.trainable = False
# lefordítjuk a modelt (fontos, hogy ezt a rétegek befagyasztása után csináljuk"
# mivel két osztályunk van, ezért bináris keresztentrópia költségfüggvényt használunk
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
model.fit(train_images, labels_onehot, batch_size=8, nb_epoch=10, validation_split=0.2, callbacks=[history])

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 4s - loss: 2.7504 - acc: 0.2727 - val_loss: 4.2366 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 0s - loss: 0.3868 - acc: 1.0000 - val_loss: 6.1345 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 0s - loss: 0.0604 - acc: 1.0000 - val_loss: 9.8033 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 0s - loss: 0.2326 - acc: 0.9091 - val_loss: 13.1851 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 0s - loss: 0.3315 - acc: 0.9091 - val_loss: 15.1209 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 0s - loss: 0.0425 - acc: 1.0000 - val_loss: 15.4690 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 0s - loss: 0.2692 - acc: 0.9091 - val_loss: 15.1603 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 0s - loss: 4.7105e-04 - ac

In [67]:
# ehhez először nézzük meg a háló felépítését
print("Az Inception V3 konvolúciós rétegei:")
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

Az Inception V3 konvolúciós rétegei:
0 input_2
1 convolution2d_95
2 batchnormalization_95
3 convolution2d_96
4 batchnormalization_96
5 convolution2d_97
6 batchnormalization_97
7 maxpooling2d_4
8 convolution2d_98
9 batchnormalization_98
10 convolution2d_99
11 batchnormalization_99
12 maxpooling2d_5
13 convolution2d_103
14 batchnormalization_103
15 convolution2d_101
16 convolution2d_104
17 batchnormalization_101
18 batchnormalization_104
19 averagepooling2d_11
20 convolution2d_100
21 convolution2d_102
22 convolution2d_105
23 convolution2d_106
24 batchnormalization_100
25 batchnormalization_102
26 batchnormalization_105
27 batchnormalization_106
28 mixed0
29 convolution2d_110
30 batchnormalization_110
31 convolution2d_108
32 convolution2d_111
33 batchnormalization_108
34 batchnormalization_111
35 averagepooling2d_12
36 convolution2d_107
37 convolution2d_109
38 convolution2d_112
39 convolution2d_113
40 batchnormalization_107
41 batchnormalization_109
42 batchnormalization_112
43 batchnorma

In [68]:
# majd a hálónak csak az első 172 rétegét fagyasztjuk, a többit pedig engedjük tanulni
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

In [69]:
# ez után újra le kell fordítanunk a hálót, hogy most már az Inception V3 felsőbb rétegei tanuljanak
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# és ismét indítunk egy tanítást, ezúttal nem csak az előrecsatolt rétegek,
# hanem az Inception V3 felső rétegei is tovább tanulnak
model.fit(train_images, labels_onehot,  batch_size=16, nb_epoch=10, validation_split=0.2, callbacks=[history])
print("Tanítás vége.")

Train on 11 samples, validate on 3 samples
Epoch 1/10
11/11 [==============================] - 7s - loss: 6.6295e-04 - acc: 1.0000 - val_loss: 5.7277 - val_acc: 0.0000e+00
Epoch 2/10
11/11 [==============================] - 5s - loss: 6.6149e-04 - acc: 1.0000 - val_loss: 5.7095 - val_acc: 0.0000e+00
Epoch 3/10
11/11 [==============================] - 5s - loss: 6.5891e-04 - acc: 1.0000 - val_loss: 5.7027 - val_acc: 0.0000e+00
Epoch 4/10
11/11 [==============================] - 5s - loss: 6.5601e-04 - acc: 1.0000 - val_loss: 5.6913 - val_acc: 0.0000e+00
Epoch 5/10
11/11 [==============================] - 5s - loss: 6.5085e-04 - acc: 1.0000 - val_loss: 5.6730 - val_acc: 0.0000e+00
Epoch 6/10
11/11 [==============================] - 5s - loss: 6.4653e-04 - acc: 1.0000 - val_loss: 5.6660 - val_acc: 0.0000e+00
Epoch 7/10
11/11 [==============================] - 5s - loss: 6.4125e-04 - acc: 1.0000 - val_loss: 5.6654 - val_acc: 0.0000e+00
Epoch 8/10
11/11 [==============================] - 5s

In [81]:
predicted_values = model.predict(np.asarray(train_images)).shape

In [79]:
max_pred = np.zeros(predicted_values.shape, dtype=int)
festok = []
for i, pred in enumerate(predicted_values):
    d = np.argmax(pred)
    max_pred[i, d] = 1
    festok.append(d)
festok

[3, 7, 4, 13, 9, 8, 2, 1, 12, 5, 6, 12, 1, 3]

In [80]:
encoder.inverse_transform(festok)

array(['5b6a7be5ffc6a27b91bd3210ffa2e088',
       'cc47068929413a16aa707faefbdf4b70',
       '69904cf890070e9593a566394d5dece4',
       'f44205b1eb2de981de766e0688f8cbac',
       'da9ab2081b197129eeb91477d239be00',
       'd500fe452aef7a6f90de16197a9670bf',
       '485d901dc4df30b128bf01cb6e229767',
       '3f93b217bd0dbf874f973958f1eb6df4',
       'f14a3a6cc3112c9e92bc6c33c88eb264',
       '8e441c5899bf3d2f3b2c493e62fb92bf',
       'c56bcab4b317984013ebef5d3c4b5906',
       'f14a3a6cc3112c9e92bc6c33c88eb264',
       '3f93b217bd0dbf874f973958f1eb6df4',
       '5b6a7be5ffc6a27b91bd3210ffa2e088'], 
      dtype='<U32')

In [73]:
# vissza kéne alakítani valahogy...
ecovered_X = np.array([ohc.active_features_[col] for col in out.sorted_indices().indices])\
.reshape(n_samples, n_features) - ohc.feature_indices_[:-1]

NameError: name 'out' is not defined